In [1]:
import requests
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context # set certificate for scraper

## Scrape S&P500 stocks summary table

In [2]:
payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp_500_summary = payload[0] # two tables on this page, go for the first one

In [3]:
sp_500_summary

,Symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,NaN
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet Holdings,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,NaN
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,NaN


In [4]:
sp_500_summary.to_csv("stock_tables/sp500_summary.csv", index=False) #save to a csv file

## Scrape S&P500 stocks by cap

In [5]:
from bs4 import BeautifulSoup

In [6]:
URL = "https://www.slickcharts.com/sp500"
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}
page = requests.get(URL, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('table', {"class":"table table-hover table-borderless table-sm"})
table_rows = table.find_all('tr')

column_names = [d.text for d in table_rows[0].find_all('th')]
column_names[4] = column_names[4].split("\xa0")[-1]
column_names[0] = "Rank"

In [7]:
table = []
for row in table_rows[1:]:
    cells = [d.text for d in row.find_all('td')]

    cells[4] = cells[4].split("\xa0")[-1]
    cells[6] = cells[6][1:-2]
    table.append(cells)

In [8]:
sp500_by_cap = pd.DataFrame(table, columns=column_names, dtype=float)
sp500_by_cap['Rank'] = sp500_by_cap['Rank'].astype(int)

In [9]:
sp500_by_cap

,Rank,Company,Symbol,Weight,Price,Chg,% Chg
0,1,Microsoft Corporation,MSFT,6.152096,207.78,-5.89,-2.76
1,2,Apple Inc.,AAPL,5.938255,382.01,-1.67,-0.44
2,3,Amazon.com Inc.,AMZN,5.094660,"3,101.00",-99.00,-3.09
3,4,Facebook Inc. Class A,FB,2.239202,239.55,-5.52,-2.25
4,5,Alphabet Inc. Class A,GOOGL,1.754667,"1,513.00",-26.01,-1.69
...,...,...,...,...,...,...,...
500,501,Gap Inc.,GPS,0.009494,11.20,-0.45,-3.86
501,502,Under Armour Inc. Class A,UAA,0.006537,9.30,-0.22,-2.31
502,503,Under Armour Inc. Class C,UA,0.006321,8.40,-0.23,-2.67
503,504,News Corporation Class B,NWS,0.005593,11.96,-0.02,-0.17


In [10]:
sp500_by_cap.to_csv("stock_tables/sp500_by_cap.csv",index=False)